In [1]:

import pandas as pd
import numpy as np 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

## Mounting the google drive to google colab

ModuleNotFoundError: No module named 'google'

In [ ]:
#root_path = '/gdrive/My Drive/Collab_files_GL/bank.csv'
dataset1 = pd.read_csv('gdrive/My Drive/Collab_files_GL/bank.csv')
dataset1

In [ ]:
dataset1.shape

In [ ]:
dataset1.describe()

In [ ]:
dataset1.columns

# 14 columns are present on the dataset 

In [ ]:
dataset1.isnull().sum()

# the column is  not having any null values 

In [ ]:
dataset1.dtypes

# Out of that 14 columns,  three columns are object datatypes 
# 2 - float64 datatypes 
# 9 - int64 datatypes  


In [ ]:
dataset1.nunique()

# Counting the total unique values on all the columns of  dataframe 
# From the below code, 
   # 1) Geography can act as categorical 
   # 2) Gender can act as categorical
   # 3) HasCrCard can act as categorical
   # 4) IsActiveMemeber act as categorical
   # 5) Exited can act as categorical

In [ ]:
dataset1.Gender.unique()

In [ ]:
dataset1.Gender.value_counts()

In [ ]:
dataset1.Geography.unique()

In [ ]:
dataset1.Geography.value_counts()

In [ ]:
dataset1.Exited.unique()

In [ ]:
dataset1.Exited.value_counts()

In [ ]:
dataset1.NumOfProducts.unique()

In [ ]:
dataset1.NumOfProducts.value_counts()

In [ ]:
dataset1.Tenure.unique()

In [ ]:
dataset1.Tenure.value_counts()

In [ ]:
dataset1.Age.value_counts()

In [ ]:
dataset1.Balance.value_counts()
median_of_balance = dataset1['Balance'].median(skipna=True)

dataset1['Balance'] = dataset1['Balance'].replace(0,median_of_balance)
# The value of 0.0 presence in 3617 rows 

In [ ]:
dataset1.Balance.value_counts()

In [ ]:
dataset1.EstimatedSalary.value_counts()

In [ ]:
import seaborn as sns 
#sns.pairplot(data =  dataset1, diag_kind ='kde' )

In [ ]:
dataset1.corr()

In [ ]:
# Dropping Row number and Customer ID column 
dataset2 = dataset1.drop(labels = ['RowNumber','CustomerId','Surname'],axis=1)
dataset2

In [ ]:
dataset2.corr()

In [ ]:
# sns.pairplot(data = dataset2,diag_kind = 'kde')

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# Converting the Sex into nominal categorical variables
# One of the ways of treating the categorical variables 
dataset2['Gender'] = encoder.fit_transform(dataset2['Gender'].astype('str'))

# What if, one hot encoding is being done for Gender and find their correlation status 

#dataset4  = pd.get_dummies(dataset2, columns = ['Gender'])

In [ ]:
dataset2.corr()
#dataset4.corr()   # Dataset for one hot encoded Gender column 

In [ ]:
dataset2

In [ ]:
# sns.pairplot(data= dataset2, hue = 'Exited', diag_kind='kde')

In [ ]:
# Study between the Exited column and the other columns 

import matplotlib.pyplot as plt 


In [ ]:

# Let us do one Hot encoding for the Geography column 
#dataset3 = pd.get_dummies(dataset2, columns = ['Geography'])
# dataset3

# Doing the  One Hot Encoding had brought a drastic decrease in the accuracy


dataset2['Geography'] = encoder.fit_transform(dataset2['Geography'].astype('str'))
dataset3 = dataset2

In [ ]:
#dataset2.columns
dataset3.columns

In [ ]:
 dataset3.columns
 #dataset2.shape

In [ ]:
# sns.box(dataset3)

In [ ]:
# Now, treating the Balance column since they are having the excess 0.0 values in them 
sns.boxplot(x = 'Balance',data= dataset3)


In [ ]:
def outlier_treatment(data1):
 sorted(data1)
 Q1,Q3 = np.percentile(data1 , [25,75])
 IQR = (Q3 - Q1)
 lower_range = (Q1 - (1.5 * IQR))
 upper_range = (Q3 + (1.5 * IQR))
 return lower_range,upper_range


lbound,ubound = outlier_treatment(dataset3.Balance)
print(lbound)
print(ubound)
dataset3[(dataset3.Balance < lbound) | (dataset3.Balance > ubound)]
dataset3.drop(dataset3[ (dataset3.Balance > ubound) | (dataset3.Balance < lbound) ].index , inplace=True)

In [ ]:
sns.boxplot(x = 'Balance',data= dataset3)

In [ ]:
sns.boxplot(x= 'EstimatedSalary', data =  dataset3)

In [ ]:
dataset3_feature = dataset3.drop(labels =['Exited','Gender','Balance'], axis = 1 )
dataset3_target  = dataset3['Exited']

In [ ]:
dataset3_feature

In [ ]:
dataset3_target

In [ ]:
from sklearn.model_selection import train_test_split 
import tensorflow as tf
tf.set_random_seed(42)


In [ ]:
tf.__version__

In [ ]:
import keras

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(dataset3_feature,dataset3_target, test_size=0.30 , random_state = 5)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
# 5. Normalize the train and test data 

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train  = ss.fit_transform(X_train)
X_test  = ss.fit_transform(X_test)

In [ ]:
X_train  # Array format

In [ ]:
X_test  # Array format

In [ ]:
# 6. Initialize & build the model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10,input_dim = 8, activation = 'relu'))
model.add(tf.keras.layers.Dense(8 ,activation = 'relu'))
model.add(tf.keras.layers.Dense(8 ,activation = 'relu'))
model.add(tf.keras.layers.Dense(8 ,activation = 'relu'))
model.add(tf.keras.layers.Dense(1 ,activation = 'sigmoid'))

# Model building is being done above with three Dense layers as 1) Input layers with Softmax activation function 
# 2) Middle Layer with sigmoid activation function 3 ) Output layer with sigmoid activation function 

# The input layer has 10 nodes with input as 12 features
# The middle layer has 12 nodes 
# the last layer has only one nodes 


from keras import optimizers

# compilation of the model 
adam = keras.optimizers.Adam(lr=0.0001)  
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train,y_train,validation_data = (X_test,y_test), epochs = 50, batch_size= X_train.shape[0])

#When trying with the softmax, the accuracy value increases from 0.199 to 0.81
# when trying with the relu, its performance is low when compared with softmax





In [ ]:
# # Doing the normalizatio by adding Batch Normalization after the Input layer and finding the accuray

# model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Dense(10,input_dim = 10, activation = 'softmax'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dense(10 ,activation = 'softmax'))
# model.add(tf.keras.layers.Dense(1 ,activation = 'sigmoid'))


In [ ]:
# model.compile(optimizer = 'sgd', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# model.fit(X_train,y_train,validation_data = (X_test,y_test), epochs = 50, batch_size= X_train.shape[0])

# # After the adding the Batch Normalization layer after the first input layer, the accuracy value doesnt show increase from its preformed 80.10 value 

In [ ]:
# # Another try

# model1 = tf.keras.models.Sequential()
# model1.add(tf.keras.layers.Dense(10,input_dim = 10, activation = 'relu'))
# model1.add(tf.keras.layers.Dense(10 ,activation = 'relu'))
# model1.add(tf.keras.layers.Dense(1 ,activation = 'relu'))

In [ ]:
# compilation of the model 

# model1.compile(optimizer = 'sgd', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# model1.fit(X_train,y_train,validation_data = (X_test,y_test), epochs = 50, batch_size= X_train.shape[0])

In [ ]:
# #Consolidating the above 

# #dataset3 = pd.get_dummies(dataset2, columns = ['Gender'])
# #dataset3

# # Let us do one Hot encoding for the Geography column 
# # dataset3 = pd.get_dummies(dataset2, columns = ['Geography'])
# # dataset3

# # Doing the  One Hot Encoding had brought a drastic decrease in the accuracy


# dataset4['Geography'] = encoder.fit_transform(dataset2['Geography'].astype('str'))

# dataset4_feature = dataset4.drop(labels =['Exited'], axis = 1 )
# dataset4_target  = dataset4['Exited']
# X_train, X_test,y_train, y_test = train_test_split(dataset4_feature,dataset4_target, test_size=0.30 , random_state = 5)
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# X_train  = ss.fit_transform(X_train)
# X_test  = ss.fit_transform(X_test)

# model2 = tf.keras.models.Sequential()
# model2.add(tf.keras.layers.Dense(10,input_dim = 11, activation = 'softmax'))
# #model2.add(tf.keras.layers.BatchNormalization())
# model2.add(tf.keras.layers.Dense(11,activation = 'softmax'))
# model2.add(tf.keras.layers.Dense(1 ,activation = 'sigmoid'))
# model2.compile(optimizer = 'sgd', loss='binary_crossentropy', metrics = ['accuracy'])
# model2.fit(X_train,y_train,validation_data = (X_test,y_test), epochs = 50, batch_size= X_train.shape[0])

# # The accuracy is not increasing even when dropping the Gender and Geography columns 

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict_classes(X_test)
final = confusion_matrix(y_test,y_pred)

In [ ]:
final